In [186]:
import numpy as np
import pandas as pd
import pdb

In [8]:
pwd

'/Users/mingyuanma/Desktop/hai-cooking/analysis new'

In [9]:
s1 = pd.read_csv("../data/Archive/combined_phase2_s1.csv")
t1 = pd.read_csv("../data/Archive/combined_phase2_t1.csv")

In [10]:
import itertools
from tqdm import tqdm

def find_string_combinations(arr):
    combinations = list(itertools.combinations(arr, 2))
    return combinations

arr = ["chef_chop", "chef_cook", "chef_plate", 
       "souchef_chop", "souchef_cook", "souchef_plate",
      "server_chop", "server_cook", "server_plate"]

In [188]:
def frequency_dictionary(df, spans=3):
    worker_id = [1,2,3]
    tasks_id = [1,2,3]
    steps = spans - 1

    df = df.join(pd.get_dummies(df["taskId"]))
    df = df.rename(columns={0:"idle_2", 1:"chop", 2:"cook", 3:"plate"})

    df = df.join(pd.get_dummies(df["workerId"]))
    warmup = df[df["round"] <= 2]
    warmup = warmup.rename(columns={0:"idle", 1:"chef", 2:"souchef", 3:"server"})
    start = df[df["round"] >= 3]
    start = start.rename(columns={0:"idle", 1:"chef", 2:"server", 3:"souchef"})
    start = start.iloc[:, [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,16,15]]
    df = pd.concat([warmup, start],axis=0)

    df["chef_chop"] = df["chef"] & df["chop"]
    df["chef_cook"] = df["chef"] & df["cook"]
    df["chef_plate"] = df["chef"] & df["plate"]

    df["souchef_chop"] = df["souchef"] & df["chop"]
    df["souchef_cook"] = df["souchef"] & df["cook"]
    df["souchef_plate"] = df["souchef"] & df["plate"]

    df["server_chop"] = df["server"] & df["chop"]
    df["server_cook"] = df["server"] & df["cook"]
    df["server_plate"] = df["server"] & df["plate"]

    arr = ["chef_chop", "chef_cook", "chef_plate", 
           "souchef_chop", "souchef_cook", "souchef_plate",
          "server_chop", "server_cook", "server_plate", "idle"]

    storage_final = {}
    dat = df.copy()
    for i in tqdm(np.arange(1,7)):
        df = df[df["round"] == i]
        feature_panels = find_string_combinations(arr)
        storage = {}
        for responser in df["ResponseId"].unique():
            sub_df = df[df["ResponseId"] == responser]
            for t in np.sort(sub_df["tick"].unique())[:-steps]:
                sub_df = sub_df[(sub_df["tick"] >= t) & (sub_df["tick"] <= t+steps)]
                for feature_panel in feature_panels:
                    feature_a = feature_panel[0]
                    feature_b = feature_panel[1]
                    if np.sum(sub_df[feature_a]) > 0 and np.sum(sub_df[feature_b]) > 0:
                        feature = feature_a + "_" + feature_b
                        count = min(np.sum(sub_df[feature_a]), np.sum(sub_df[feature_b]))
                        if not storage.get(feature):
                            storage[feature] = np.array([count])
                        else: 
                            storage[feature] += count
        for k, v in storage.items():
            storage[k] = v / len(df["ResponseId"].unique())
        df = dat
        storage_final[i] = storage
    return storage_final      

In [189]:
s1_frequency = frequency_dictionary(s1, 6)
t1_frequency = frequency_dictionary(t1, 6)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:36<00:00,  6.08s/it]


In [190]:
def dic_to_df(dic):
    round1 = pd.DataFrame(dic[1]).T.reset_index()
    round2 = pd.DataFrame(dic[2]).T.rename(columns={0:1}).reset_index()
    df = pd.merge(round1,round2, how='outer')

    round3 = pd.DataFrame(dic[3]).T.rename(columns={0:2}).reset_index()
    df = pd.merge(df,round3, how='outer')

    round4 = pd.DataFrame(dic[4]).T.rename(columns={0:3}).reset_index()
    df = pd.merge(df,round4, how='outer')

    round5 = pd.DataFrame(dic[5]).T.rename(columns={0:4}).reset_index()
    df = pd.merge(df,round5, how='outer')

    round6 = pd.DataFrame(dic[6]).T.rename(columns={0:5}).reset_index()
    df = pd.merge(df,round6, how='outer')

    df = df.fillna(0).rename(columns = {"index":"strategy", 
                                  0:"round 1",
                                  1:"round 2",
                                  2:"round 3",
                                  3:"round 4",
                                  4:"round 5",
                                  5:"round 6"})
    return df

In [191]:
final = pd.merge(dic_to_df(s1_frequency), dic_to_df(t1_frequency), on="strategy")
s1_final = final.iloc[:, 0:7].rename(columns={
    "round 1_x":"round 1","round 2_x":"round 2","round 3_x":"round 3",
    "round 4_x":"round 4","round 5_x":"round 5","round 6_x":"round 6",
})
t1_final = final.iloc[:, [0,7,8,9,10,11,12]].rename(columns={
    "round 1_y":"round 1","round 2_y":"round 2","round 3_y":"round 3",
    "round 4_y":"round 4","round 5_y":"round 5","round 6_y":"round 6",
})
dif = s1_final.iloc[:,1:] - t1_final.iloc[:,1:]
dif

,round 1,round 2,round 3,round 4,round 5,round 6
0,0.019837,-0.046309,0.077433,-0.097808,0.210532,0.223012
1,-0.029109,0.016629,0.000000,0.000000,0.000000,0.000000
2,-0.177022,-0.032989,0.000000,0.000000,0.000000,0.000000
3,0.033661,0.072478,0.039725,-0.102360,0.147829,0.152146
4,-0.227513,-0.180684,-0.099807,-0.390073,-0.747258,-0.631293
5,0.059444,-0.046594,0.000000,0.000000,0.000000,0.000000
6,0.005140,0.249131,0.000000,0.000000,0.000000,0.000000
7,-0.024523,0.125036,0.229966,0.354917,1.029075,0.995784
8,0.136542,-0.364307,0.092887,0.060015,0.878139,0.931167
9,-0.053296,0.133132,0.000000,0.000000,0.000000,0.000000


In [192]:
dif.iloc[[7,8,4,14,15,16,20],:]

,round 1,round 2,round 3,round 4,round 5,round 6
7,-0.024523,0.125036,0.229966,0.354917,1.029075,0.995784
8,0.136542,-0.364307,0.092887,0.060015,0.878139,0.931167
4,-0.227513,-0.180684,-0.099807,-0.390073,-0.747258,-0.631293
14,0.012178,0.137146,0.378567,0.493458,1.347644,1.150920
15,0.012530,-0.014160,-0.164407,-0.365734,-0.719610,-0.854338
16,0.012732,0.061141,-0.060116,-0.095339,-0.340539,-0.348770
20,0.009893,0.140287,-0.371160,-0.569799,-1.096347,-1.338070


In [193]:
s1_final.iloc[[7,8,4,14,15,16,20],:]

,strategy,round 1,round 2,round 3,round 4,round 5,round 6
7,chef_cook_server_chop,0.979592,0.808163,3.559184,2.877551,2.967347,3.012245
8,chef_cook_idle,4.910204,4.763265,4.714286,4.208163,4.306122,4.363265
4,chef_chop_idle,1.142857,1.453061,3.126531,2.893878,2.791837,2.702041
14,server_chop_idle,1.020408,0.836735,5.004082,4.044898,4.261224,4.163265
15,chef_chop_server_cook,0.477551,0.220408,0.975510,1.436735,1.432653,1.302041
16,chef_cook_server_cook,1.453061,0.571429,0.804082,1.583673,1.375510,1.375510
20,server_cook_idle,2.297959,0.955102,1.595918,2.730612,2.706122,2.575510


In [194]:
t1_final.iloc[[7,8,4,14,15,16,20],:]

,strategy,round 1,round 2,round 3,round 4,round 5,round 6
7,chef_cook_server_chop,1.004115,0.683128,3.329218,2.522634,1.938272,2.016461
8,chef_cook_idle,4.773663,5.127572,4.621399,4.148148,3.427984,3.432099
4,chef_chop_idle,1.370370,1.633745,3.226337,3.283951,3.539095,3.333333
14,server_chop_idle,1.008230,0.699588,4.625514,3.551440,2.913580,3.012346
15,chef_chop_server_cook,0.465021,0.234568,1.139918,1.802469,2.152263,2.156379
16,chef_cook_server_cook,1.440329,0.510288,0.864198,1.679012,1.716049,1.724280
20,server_cook_idle,2.288066,0.814815,1.967078,3.300412,3.802469,3.913580
